In [25]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import quaternion
import sympy
from sympy.abc import x,y,z
from sympy.utilities.codegen import codegen

## Manage files

In [26]:
FILE_NAME = "code_gen"
with open(FILE_NAME+".c","w") as f:
    pass

## Generate C code for error

In [27]:
from cmath import sqrt


theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

jn = j1.cross(j2)
jn_norm = jn/(sympy.sqrt(jn.dot(jn)))
error = omegaR.dot(jn_norm)
error = sympy.simplify(error)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_sph_11', error), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

error

(omega_x*(sin(rho_1)*sin(theta_1)*cos(theta_2) - sin(rho_2)*sin(theta_2)*cos(theta_1)) - omega_y*(sin(theta_1)*cos(rho_1)*cos(theta_2) - sin(theta_2)*cos(rho_2)*cos(theta_1)) - omega_z*sin(theta_1)*sin(theta_2)*sin(rho_1 - rho_2))/sqrt((sin(rho_1)*sin(theta_1)*cos(theta_2) - sin(rho_2)*sin(theta_2)*cos(theta_1))**2 + (sin(theta_1)*cos(rho_1)*cos(theta_2) - sin(theta_2)*cos(rho_2)*cos(theta_1))**2 + sin(theta_1)**2*sin(theta_2)**2*sin(rho_1 - rho_2)**2)

In [28]:
# Declare symbols
jx1,jy1,jz1 = sympy.symbols('j_x1 j_y1 j_z1')
jx2,jy2,jz2 = sympy.symbols('j_x2 j_y2 j_z2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')

# Rotation vectors
j1 = sympy.Matrix([jx1, jy1, jz1 ])
j2 = sympy.Matrix([jx2, jy2, jz2 ])

# Relative angular velocity
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

# Error and derivatives compute
error = omegaR.dot(j1.cross(j2))

[(c_name, c_code), (h_name, c_header)] = codegen(('error_xyz', error), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

## Generate C code for spherical transformations

### Vector to spherical

In [29]:
# atan2(sqrt(vector[0]*vector[0]+vector[1]*vector[1]) , vector[2]);
sph_theta = sympy.atan2(sympy.sqrt(x**2 + y**2),z)
sph_rho   = sympy.atan2(y,x)
sph = sympy.Matrix(2,1, [
    sph_theta, sph_rho
])

[(c_name, c_code), (h_name, c_header)] = codegen(('spherical_1', sph), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

sph_theta = sympy.atan2(sympy.sqrt(z**2 + y*y),x)
sph_rho   = sympy.atan2(y,z)
sph = sympy.Matrix(2,1, [
    sph_theta, sph_rho
])

[(c_name, c_code), (h_name, c_header)] = codegen(('spherical_2', sph), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

### Spherical to 3D vector

In [30]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
[(c_name, c_code), (h_name, c_header)] = codegen(('spherical_2_vec_1', j1), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

j1 = sympy.Matrix([
        sympy.cos(theta1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.sin(theta1)*sympy.cos(rho1)
    ])
[(c_name, c_code), (h_name, c_header)] = codegen(('spherical_2_vec_2', j1), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

## Generate C code for error and derivatives

In [31]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')

j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

jn = j1.cross(j2)
jn_norm = jn/(sympy.sqrt(jn.dot(jn)))
error = omegaR.dot(jn_norm)
error = sympy.simplify(error)
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
J_col = sympy.simplify(J_col)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_11', J_col), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

In [32]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')

j1 = sympy.Matrix([
        sympy.cos(theta1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.sin(theta1)*sympy.cos(rho1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

jn = j1.cross(j2)
jn_norm = jn/(sympy.sqrt(jn.dot(jn)))
error = omegaR.dot(jn_norm)
error = sympy.simplify(error)
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
J_col = sympy.simplify(J_col)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_21', J_col), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

In [33]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')

j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.cos(theta2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.sin(theta2)*sympy.cos(rho2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

jn = j1.cross(j2)
jn_norm = jn/(sympy.sqrt(jn.dot(jn)))
error = omegaR.dot(jn_norm)
error = sympy.simplify(error)
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
J_col = sympy.simplify(J_col)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_12', J_col), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)

In [34]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')

j1 = sympy.Matrix([
        sympy.cos(theta1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.sin(theta1)*sympy.cos(rho1)
    ])
j2 = sympy.Matrix([
        sympy.cos(theta2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.sin(theta2)*sympy.cos(rho2)
    ])
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

jn = j1.cross(j2)
jn_norm = jn/(sympy.sqrt(jn.dot(jn)))
error = omegaR.dot(jn_norm)
error = sympy.simplify(error)
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
J_col = sympy.simplify(J_col)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_derivatives_22', J_col), "C99", FILE_NAME, header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)